# Particle physics results

## Setup

In [1]:
import sys
import numpy as np
from sklearn.metrics import roc_auc_score
from sklearn.neighbors import KernelDensity
from itertools import product


## Load results

In [2]:
n_runs = 10
n_chains = 4
n_trueparams = 3

In [3]:
algo_filenames = []
algo_additionals = []
algo_labels = []
algo_dividers = []
algo_dims = []

def add_algo(filename, add, label, dim=""):
    algo_filenames.append(filename)
    algo_additionals.append(add)
    algo_labels.append(label)
    algo_dims.append(dim)
    
    
def add_divider():
    algo_dividers.append(len(algo_filenames))

add_algo("flow", "_may", r"\af{}", "40d")
add_algo("pie", "_conditionalmanifold_may", r"\pie{} (original)", "40d")
add_algo("pie", "_may", r"\pie{} (unconditional manifold)", "40d")
add_algo("mf", "_sequential_may", r"\mfmfmd{sequential }", "40d")
add_algo("mf", "_alternate_may", r"\mfmfmd{alternating }", "40d")
add_algo("emf", "_sequential_may", r"\mfmfemd{sequential }", "40d")
add_algo("emf", "_alternate_may", r"\mfmfemd{alternating }", "40d")

add_divider()

add_algo("flow", "_scandal_may", r"\afscandal{}", "40d")
add_algo("pie", "_conditionalmanifold_scandal_may", r"\piescandal{, original}", "40d")
add_algo("pie", "_scandal_may", r"\piescandal{, unconditional man.}", "40d")
add_algo("mf", "_sequential_scandal_may", r"\mfmfscandal{sequential }", "40d")
add_algo("mf", "_alternate_scandal_may", r"\mfmfscandal{alternating }", "40d")
add_algo("emf", "_sequential_scandal_may", r"\mfmfescandal{sequential }", "40d")
add_algo("emf", "_alternate_scandal_may", r"\mfmfescandal{alternating }", "40d")

add_divider()

add_algo("flow", "_may", "2D summaries (AF)", "2d")
add_algo("flow", "_scandal_may", "2D summaries (SCANDAL)", "2d")
add_algo("alices", "_may", "ALICES")

n_algos = len(algo_filenames)


In [4]:
def load(name, shape, numpyfy=True, chains=1, result_dir="../data/results"):
    all_results = []
    
    for algo_filename, algo_add, algo_dim in zip(algo_filenames, algo_additionals, algo_dims):
        algo_results = []
            
        for run in range(n_runs):
            run_str = "" if run == 0 else "_run{}".format(run)
            
            for trueparam in range(n_trueparams):
                trueparam_str = "" if trueparam == 0 else "_trueparam{}".format(trueparam)

                try:
                    this_result = np.load(
                        "{}/{}_{}_lhc{}{}{}_{}{}.npy".format(
                            result_dir, algo_filename, "2" if algo_dim == "2d" else "14",
                            algo_dim, algo_add, run_str, name, trueparam_str
                        )
                    )
                    if (not numpyfy) or (shape is None) or np.product(this_result.shape) == np.product(shape):
                        algo_results.append(this_result.reshape(shape))
                    else:
                        algo_results.append(np.nan*np.ones(shape))

                except FileNotFoundError as e:
                    # print(e)
                    if shape is None:
                        algo_results.append(None)
                    else:
                        algo_results.append(np.nan*np.ones(shape))
            
        all_results.append(algo_results)
    
    if numpyfy:
        all_results = np.array(all_results, dtype=np.float)
        all_results = all_results.reshape([all_results.shape[0], n_runs, n_trueparams] + list(shape))
        
    return all_results


model_gen_x = load("samples", None, numpyfy=False)
model_gen_closure = load("samples_manifold_distance", (10000,))
model_test_reco_error = load("model_reco_error_test", (1000,))



In [5]:
def load_mcmc(name, shape, numpyfy=True, result_dir="../data/results"):
    all_results = []
    
    for algo_filename, algo_add, algo_dim in zip(algo_filenames, algo_additionals, algo_dims):
        algo_results = []
            
        for run in range(n_runs):
            run_str = "" if run == 0 else "_run{}".format(run)
            
            for trueparam in range(n_trueparams):
                trueparam_str = "" if trueparam == 0 else "_trueparam{}".format(trueparam)
            
                for chain in range(n_chains):
                    chain_str = "" if chain == 0 else "_chain{}".format(chain)

                    try:
                        this_result = np.load(
                            "{}/{}_{}_lhc{}{}{}_{}{}{}.npy".format(
                                result_dir, algo_filename, "2" if algo_dim == "2d" else "14",
                                algo_dim, algo_add, run_str, name, trueparam_str, chain_str
                            )
                        )
                        if (not numpyfy) or (shape is None) or np.product(this_result.shape) == np.product(shape):
                            algo_results.append(this_result.reshape(shape))
                        else:
                            algo_results.append(np.nan*np.ones(shape))

                    except FileNotFoundError as e:
                        # print(e)
                        if shape is None:
                            algo_results.append(None)
                        else:
                            algo_results.append(np.nan*np.ones(shape))
            
        all_results.append(algo_results)
    
    all_results = np.array(all_results, dtype=np.float)
    all_results = all_results.reshape([all_results.shape[0], n_runs, n_trueparams, n_chains] + list(shape))
        
    return all_results


model_posterior_samples = load_mcmc("posterior_samples", (750, 2,))
model_posterior_samples.shape  # (algo, run, true param id, chain, sample, theta component)


(17, 10, 3, 4, 750, 2)

## Calculate metrics

In [6]:
model_gen_mean_closure = np.mean(model_gen_closure, axis=(2,3))
model_gen_mean_closure.shape


(17, 10)

In [7]:
model_mean_reco_error = np.mean(model_test_reco_error, axis=(2,3))
model_mean_reco_error.shape


(17, 10)

In [8]:
bandwidth = 0.1
true_param_points = np.array([[0.,0.], [0.5, 0.], [-1., -1.]])

model_true_log_posteriors = []

for algo, run, trueparam in product(range(n_algos), range(n_runs), range(n_trueparams)):
    mcmcs = model_posterior_samples[algo, run, trueparam].reshape((-1, 2))
    mcmcs = mcmcs[np.all(np.isfinite(mcmcs), axis=-1)]
    
    if len(mcmcs) == 0:
        model_true_log_posteriors.append(np.nan)
        continue
        
    kde = KernelDensity(kernel="gaussian", bandwidth=bandwidth)
    kde.fit(mcmcs)
    model_true_log_posteriors.append(kde.score(true_param_points[trueparam].reshape((1, 2))))

model_true_log_posteriors = np.mean(np.array(model_true_log_posteriors).reshape((n_algos, n_runs, n_trueparams)), axis=-1)
model_true_log_posteriors.shape


(17, 10)

## Outlier removal

In [9]:
def mean_err_without_outliers(data, remove=1):
    shape = list(data.shape)[:-1]
    data.reshape((-1, data.shape[-1]))
    
    means, errors = [], []
    
    for data_ in data:
        data_ = data_[np.isfinite(data_)]
        if not len(data_) > 0:
            means.append(np.nan)
            errors.append(np.nan)
            continue
            
        if len(data_) > 2*remove + 1:
            for _ in range(remove):
                data_ = np.delete(data_, np.argmin(data_))
                data_ = np.delete(data_, np.argmax(data_))

        means.append(np.mean(data_))
        errors.append(np.std(data_) / len(data_)**0.5)
        
    return np.array(means).reshape(shape), np.array(errors).reshape(shape)
    
    
model_true_log_posteriors_mean, model_true_log_posteriors_std = mean_err_without_outliers(model_true_log_posteriors)
model_gen_mean_closure_mean, model_gen_mean_closure_std = mean_err_without_outliers(model_gen_mean_closure)
model_mean_reco_error_mean, model_mean_reco_error_std = mean_err_without_outliers(model_mean_reco_error)


## Best metrics

In [10]:
best_closure, best_posterior = -1, -1

best_closure = np.nanargmin(model_gen_mean_closure_mean)
print(algo_labels[best_closure])

best_reco = np.nanargmin(np.where(model_mean_reco_error_mean > 1.e-3, model_mean_reco_error_mean, np.nan))
print(algo_labels[best_reco])
    
best_posterior = np.nanargmax(model_true_log_posteriors_mean)
print(algo_labels[best_posterior])


\af{}
\mfmfmd{sequential }
\mfmfescandal{sequential }


/Users/johannbrehmer/anaconda3/envs/ml/lib/python3.6/site-packages/ipykernel_launcher.py:6: RuntimeWarning: invalid value encountered in greater
  


## Print result table

In [11]:
def print_results(
    l_label=max([len(l) for l in algo_labels]), l_mean=6, l_err=5, latex=False, after_decs=(3,3,2)
):
    # Number of digits
    l_result = l_mean + 2 + l_err
    l_total = l_label + 1 + 3 * (3 + l_result)
        
    # Divider
    empty_result = "" if latex else " "*(l_result + 1)
    col_divider = "&" if latex else "|"
    line_end = r"\\" if latex else ""
    block_divider = r"\midrule" if latex else "-"*l_total
    
    # Number formatting
    def _f(val, err, after_dec, best=False):
        if not np.any(np.isfinite(val)):
            return empty_result
        
        result = "{:>{}.{}f}".format(val, l_mean, after_dec)
        if latex and best:
            result = r"\textbf{" + result + "}"
            
        err_str = "({:>{}.{}f})".format(err, l_err, after_dec)
        if latex:
            result += r"\,\textcolor{darkgray}{" + err_str + "}"
        else:
            result += err_str
            
        result += "*" if not latex and best else " "
        
        if latex:
            result = result.replace("-", "$-{}$")
            result = result.replace("darkgray", "dark-gray")
        return result
    
    
    # Header
    print(
        "{2:<{0}.{0}s} {5} {3:>{1}.{1}s} {5} {7:>{1}.{1}s} {5} {4:>{1}.{1}s} {6}".format(
            l_label, l_result, "", "Closure", "log p", col_divider, line_end, "Reco error"
        )
    )
    print(block_divider)

    # Iterate over methods
    for i, (label, closure, closure_err, posterior, posterior_err, reco, reco_err) in enumerate(zip(
        algo_labels,
        model_gen_mean_closure_mean,
        model_gen_mean_closure_std,
        model_true_log_posteriors_mean,
        model_true_log_posteriors_std,
        model_mean_reco_error_mean,
        model_mean_reco_error_std,
    )):
        # Divider
        if i in algo_dividers:
            print(block_divider)
            
        # Print results
        print(
            "{1:<{0}.{0}s} {4} {2}{4} {6}{4} {3} {5}".format(
                l_label, label,
                _f(closure, closure_err, after_decs[0], i==best_closure),
                _f(posterior, posterior_err, after_decs[2], i==best_posterior),
                col_divider, line_end,
                _f(reco, reco_err, after_decs[1], i==best_reco),
            )
        )


In [12]:
print_results()

                                  |       Closure |    Reco error |         log p 
----------------------------------------------------------------------------------
\af{}                             |  0.014(0.002)*|  0.000(0.000) | -10.55( 3.42)  
\pie{} (original)                 |               |               |                
\pie{} (unconditional manifold)   |  0.036(0.006) |  4.599(0.243) |  -1.78( 0.14)  
\mfmfmd{sequential }              |  0.047(0.011) |  0.323(0.137)*|  -1.85( 0.14)  
\mfmfmd{alternating }             |  0.156(0.014) |  0.628(0.126) |  -2.10( 0.07)  
\mfmfemd{sequential }             |  0.056(0.010) |               |  -2.57( 0.42)  
\mfmfemd{alternating }            |  0.056(0.011) |               |  -2.42( 0.31)  
----------------------------------------------------------------------------------
\afscandal{}                      |  0.062(0.004) |  0.000(0.000) |   0.04( 0.11)  
\piescandal{, original}           |               |               |            

In [13]:
print_results(latex=True)

                                  &       Closure &    Reco error &         log p \\
\midrule
\af{}                             & \textbf{ 0.014}\,\textcolor{dark-gray}{(0.002)} &  0.000\,\textcolor{dark-gray}{(0.000)} & $-{}$10.55\,\textcolor{dark-gray}{( 3.42)}  \\
\pie{} (original)                 & & &  \\
\pie{} (unconditional manifold)   &  0.036\,\textcolor{dark-gray}{(0.006)} &  4.599\,\textcolor{dark-gray}{(0.243)} &  $-{}$1.78\,\textcolor{dark-gray}{( 0.14)}  \\
\mfmfmd{sequential }              &  0.047\,\textcolor{dark-gray}{(0.011)} & \textbf{ 0.323}\,\textcolor{dark-gray}{(0.137)} &  $-{}$1.85\,\textcolor{dark-gray}{( 0.14)}  \\
\mfmfmd{alternating }             &  0.156\,\textcolor{dark-gray}{(0.014)} &  0.628\,\textcolor{dark-gray}{(0.126)} &  $-{}$2.10\,\textcolor{dark-gray}{( 0.07)}  \\
\mfmfemd{sequential }             &  0.056\,\textcolor{dark-gray}{(0.010)} & &  $-{}$2.57\,\textcolor{dark-gray}{( 0.42)}  \\
\mfmfemd{alternating }            &  0.056\,\textcolor{dar

## Individual run results

In [14]:
l_label=max([len(l) for l in algo_labels])
l_mean=6
after_decs=2

# How to format the numbers
l_result = 3 + n_runs*l_mean + (n_runs - 1)*2
l_total = l_label + 4 + l_result

# Divider
empty_result = " "*(l_result + 1)
col_divider = "|"
line_end = ""
block_divider = "-"*l_total
    
def _f(val, after_dec, best=False):
    if not np.any(np.isfinite(val)):
        return empty_result
    result = " [{:>{}.{}f}, ".format(np.nanmean(val[0]), l_mean, after_dec)
    for i in range(1, n_runs - 1):
        result += "{:>{}.{}f}, ".format(np.nanmean(val[i]), l_mean, after_dec)
    result += "{:>{}.{}f}]".format(np.nanmean(val[-1]), l_mean, after_dec)
    result = result.replace("nan", "   ")
    result += "*" if best else " "
    return result



# Print closure results
print(
    "{2:<{0}.{0}s} {4} {3:>{1}.{1}s} {5}".format(
        l_label, l_result, "", "Closure", col_divider, line_end
    )
)
print(block_divider)

for i, (label, closure) in enumerate(zip(algo_labels, model_gen_mean_closure)):
    # Divider
    if i in algo_dividers:
        print(block_divider)
            
    # Print results
    print("{1:<{0}.{0}s} {3} {2} {4}".format(
        l_label, label, _f(closure, after_decs, i==best_closure), col_divider, line_end
    ))

print("")
print("")
print("")
    
# Print reco error results
print(
    "{2:<{0}.{0}s} {4} {3:>{1}.{1}s} {5}".format(
        l_label, l_result, "", "Closure", col_divider, line_end
    )
)
print(block_divider)

for i, (label, reco) in enumerate(zip(algo_labels, model_test_reco_error)):
    # Divider
    if i in algo_dividers:
        print(block_divider)
            
    # Print results
    print("{1:<{0}.{0}s} {3} {2} {4}".format(
        l_label, label, _f(reco, after_decs, i==best_reco), col_divider, line_end
    ))

print("")
print("")
print("")
    
# Print posterior results
print(
    "{2:<{0}.{0}s} {4} {3:>{1}.{1}s} {5}".format(
        l_label, l_result, "", "Log posterior", col_divider, line_end
    )
)
print(block_divider)

for i, (label, posterior) in enumerate(zip(algo_labels, model_true_log_posteriors)):
    # Divider
    if i in algo_dividers:
        print(block_divider)
            
    # Print results
    print("{1:<{0}.{0}s} {3} {2} {4}".format(
        l_label, label, _f(posterior, after_decs, i==best_posterior), col_divider, line_end
    ))


                                  |                                                                           Closure 
----------------------------------------------------------------------------------------------------------------------
\af{}                             |  [  0.01,   0.02,   0.01,   0.01,   0.02,   0.02,   0.01,   0.02,   0.01,   0.01]* 
\pie{} (original)                 |                                                                                    
\pie{} (unconditional manifold)   |  [  0.05,   0.02,   0.04,   0.02,   0.06,   0.01,   0.02,   0.06,   0.10,   0.02]  
\mfmfmd{sequential }              |  [  0.07,   0.01,   0.02,   0.03,   0.02,   0.21,   0.09,   0.09,   0.02,   0.03]  
\mfmfmd{alternating }             |  [  0.17,   0.09,   0.13,   0.23,   0.12,   0.04,   0.23,   0.16,   0.15,   0.19]  
\mfmfemd{sequential }             |  [  0.08,   0.13,   0.05,   0.07,   0.03,   0.10,   0.02,   0.07,   0.01,   0.02]  
\mfmfemd{alternating }            |  [  0.

/Users/johannbrehmer/anaconda3/envs/ml/lib/python3.6/site-packages/ipykernel_launcher.py:20: RuntimeWarning: Mean of empty slice
/Users/johannbrehmer/anaconda3/envs/ml/lib/python3.6/site-packages/ipykernel_launcher.py:21: RuntimeWarning: Mean of empty slice
/Users/johannbrehmer/anaconda3/envs/ml/lib/python3.6/site-packages/ipykernel_launcher.py:18: RuntimeWarning: Mean of empty slice


\mfmfmd{alternating }             |  [  0.76,   0.07,   0.83,   1.21,   0.09,   0.05,   1.55,   0.67,   0.70,   0.70]  
\mfmfemd{sequential }             |                                                                                    
\mfmfemd{alternating }            |                                                                                    
----------------------------------------------------------------------------------------------------------------------
\afscandal{}                      |  [  0.00,   0.00,   0.00,   0.00,   0.00,   0.00,   0.00,       ,       ,       ]  
\piescandal{, original}           |                                                                                    
\piescandal{, unconditional man.} |  [  3.83,   3.44,   4.12,   0.80,   3.91,   2.24,   6.76,   3.82,       ,       ]  
\mfmfscandal{sequential }         |  [  0.73,   0.75,   0.08,   0.98,   0.62,   0.13,   0.08,       ,       ,       ]  
\mfmfscandal{alternating }        |  [   